In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx

import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
adj_mx = pd.read_pickle('C:\Code\PopFrame\examples\data\model_data\_adj_mx.parquet')
towns = gpd.read_parquet('C:\Code\PopFrame\examples\data\model_data/towns.parquet')
okrugs = gpd.read_parquet('C:\Code\PopFrame\examples\data\model_data\okrugs.parquet')
rayons = gpd.read_parquet('C:\Code\PopFrame\examples\data\model_data/rayons.parquet')
territories = gpd.read_parquet('C:\Code\PopFrame\examples\data\model_data/territories.parquet')


In [3]:
from popframe.models.region import Region

region_model = Region(
  towns=towns,
  settlements=okrugs, 
  districts=rayons, 
  accessibility_matrix=adj_mx,
  territories=territories
)

In [14]:
profiles = {
    "Жилая застройка - ИЖС": {
        "criteria": {"Население": 1, "Транспорт": 2, "Экология": 4, "Соц-об": 4, "Инж инф": 3, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 1, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Малоэтажная": {
        "criteria": {"Население": 3, "Транспорт": 3, "Экология": 4, "Соц-об": 5, "Инж инф": 4, "Кул-дос-сп": 1},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Среднеэтажная": {
        "criteria": {"Население": 4, "Транспорт": 4, "Экология": 4, "Соц-об": 4, "Инж инф": 5, "Кул-дос-сп": 2},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Многоэтажная": {
        "criteria": {"Население": (4, 5), "Транспорт": 5, "Экология": 4, "Соц-об": 5, "Инж инф": 5, "Кул-дос-сп": 3},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Общественно-деловая": {
        "criteria": {"Население": 4, "Транспорт": 5, "Экология": 4, "Соц-об": 2, "Инж инф": 5, "Кул-дос-сп": 3},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Рекреационная": {
        "criteria": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Специального назначения - медицинский центр": {
        "criteria": {"Население": 4, "Транспорт": 4, "Экология": 5, "Соц-об": 0, "Инж инф": 5, "Кул-дос-сп": 0},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Специального назначения - туристический кластер": {
        "criteria": {"Население": 3, "Транспорт": 3, "Экология": 5, "Соц-об": 1, "Инж инф": 3, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Промышленная": {
        "criteria": {"Население": (3, 4), "Транспорт": 4, "Экология": 0, "Соц-об": 1, "Инж инф": 4, "Кул-дос-сп": 2},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 0, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Сельско-хозяйственная": {
        "criteria": {"Население": (2, 3), "Транспорт": 3, "Экология": (4, 5), "Соц-об": 3, "Инж инф": (2, 3), "Кул-дос-сп": 2},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Транспортная инженерная": {
        "criteria": {"Население": (1, 2), "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 2, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    }
}

territories_criteria = {
    "Лодейное Поле": {
        "Население": 2,
        "Транспорт": 3,
        "Экология": 5,
        "Соц-об": 3,
        "Инж инф": 2,
        "Кул-дос-сп": 3
    },
    "Шлиссельбург": {
        "Население": 3,
        "Транспорт": 3,
        "Экология": 4,
        "Соц-об": 4,
        "Инж инф": 4,
        "Кул-дос-сп": 4
    },
    "Сиверский": {
        "Население": 2,
        "Транспорт": 3,
        "Экология": 4,
        "Соц-об": 3,
        "Инж инф": 3,
        "Кул-дос-сп": 2
    },
    "Светогорск": {
        "Население": 2,
        "Транспорт": 3,
        "Экология": 5,
        "Соц-об": 2,
        "Инж инф": 2,
        "Кул-дос-сп": 2
    }
}


In [15]:
from popframe.method.territory_evaluation import TerritoryEvaluation

evaluation = TerritoryEvaluation(region=region_model)
# Печать значений для каждой территории
for territory, criteria in territories_criteria.items():
    print(territory)
    ranked_profiles = evaluation.calculate_potential(profiles, criteria)
    for profile, (potential, weighted_score) in ranked_profiles:
        print(f"Профиль: {profile}, Потенциал: {potential}, Соответствий важным критериям: {weighted_score}")
    print('_______________________')

Лодейное Поле
Профиль: Сельско-хозяйственная, Потенциал: 6, Соответствий важным критериям: 1
Профиль: Транспортная инженерная, Потенциал: 6, Соответствий важным критериям: 0
Профиль: Жилая застройка - ИЖС, Потенциал: 4, Соответствий важным критериям: 1
Профиль: Специального назначения - туристический кластер, Потенциал: 4, Соответствий важным критериям: 0
Профиль: Жилая застройка - Малоэтажная, Потенциал: 3, Соответствий важным критериям: 1
Профиль: Общественно-деловая, Потенциал: 3, Соответствий важным критериям: 1
Профиль: Специального назначения - медицинский центр, Потенциал: 3, Соответствий важным критериям: 0
Профиль: Промышленная, Потенциал: 3, Соответствий важным критериям: 0
Профиль: Жилая застройка - Среднеэтажная, Потенциал: 2, Соответствий важным критериям: 1
Профиль: Жилая застройка - Многоэтажная, Потенциал: 2, Соответствий важным критериям: 1
Профиль: Рекреационная, Потенциал: 6, Соответствий важным критериям: 0
_______________________
Шлиссельбург
Профиль: Жилая застрой